In [0]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import os
from sklearn.naive_bayes import MultinomialNB

In [234]:
!git clone https://github.com/Tariq60/LIAR-PLUS.git

fatal: destination path 'LIAR-PLUS' already exists and is not an empty directory.


###Binary Classification using Logistic Regression , Naive Bayes and SVM on only statement

In [0]:
df = pd.read_csv(os.getcwd()+'/LIAR-PLUS'+'/dataset'+'/train2.tsv' , sep = '\t' , header = None)
df_val = pd.read_csv(os.getcwd()+'/LIAR-PLUS'+'/dataset'+'/val2.tsv' , sep = '\t' , header = None)
df_test = pd.read_csv(os.getcwd()+'/LIAR-PLUS'+'/dataset'+'/test2.tsv' , sep = '\t' , header = None)

In [0]:
df = df.drop([0,1,4,5,6,7,8,9,10,11,12,13,14,15],axis=1)

df_val = df_val.drop([0,1,4,5,6,7,8,9,10,11,12,13,14,15],axis=1)

df_test = df_test.drop([0,1,4,5,6,7,8,9,10,11,12,13,14,15],axis=1)

df.loc[(df[2] == 'true') , 2] = 5
df.loc[(df[2] == 'mostly-true') , 2] = 4
df.loc[(df[2] == 'half-true') , 2] = 3
df.loc[(df[2] == 'barely-true') , 2] = 2
df.loc[(df[2] == 'false') , 2] = 1
df.loc[(df[2] == 'pants-fire') , 2] = 0
df.columns = ['label','text']

df['label'] = (df['label'] > 2).astype(int)

df_val.loc[(df_val[2] == 'true') , 2] = 5
df_val.loc[(df_val[2] == 'mostly-true') , 2] = 4
df_val.loc[(df_val[2] == 'half-true') , 2] = 3
df_val.loc[(df_val[2] == 'barely-true') , 2] = 2
df_val.loc[(df_val[2] == 'false') , 2] = 1
df_val.loc[(df_val[2] == 'pants-fire') , 2] = 0
df_val.columns = ['label','text']

df_val['label'] = (df_val['label'] > 2).astype(int)

df_test.loc[(df_test[2] == 'true') , 2] = 5
df_test.loc[(df_test[2] == 'mostly-true') , 2] = 4
df_test.loc[(df_test[2] == 'half-true') , 2] = 3
df_test.loc[(df_test[2] == 'barely-true') , 2] = 2
df_test.loc[(df_test[2] == 'false') , 2] = 1
df_test.loc[(df_test[2] == 'pants-fire') , 2] = 0
df_test.columns = ['label','text']

df_test['label'] = (df_test['label'] > 2).astype(int)

df = pd.concat([df,df_val])


In [0]:
labels = df['label'].tolist()
labels = np.array(labels)
labels

labels_val = df_val['label'].tolist()
labels_val = np.array(labels_val)


labels_test = df_test['label'].tolist()
labels_test = np.array(labels_test)

In [0]:
texts = df['text'].tolist()
texts_val = df_val['text'].tolist()
texts_test = df_test['text'].tolist()

cvect = CountVectorizer()
cout = cvect.fit_transform(texts)
cout_val = cvect.transform(texts_val)
cout_test = cvect.transform(texts_test)

tfidfvect = TfidfTransformer()
tout = tfidfvect.fit_transform(cout)
tout_val = tfidfvect.transform(cout_val)
tout_test = tfidfvect.transform(cout_test)

# tout = cout
# tout_val = cout_val

In [288]:
tout[0]

<1x12872 sparse matrix of type '<class 'numpy.float64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [0]:
nbmodel = MultinomialNB().fit(tout,labels)

In [0]:
pred = nbmodel.predict(tout_test)

In [291]:
cur = 0
for i in range(len(pred)):
    if pred[i] == labels_test[i]:
        cur += 1

print("Accuracy: %.2f%%" % ((cur/float(len(labels_test)) * 100)))

Accuracy: 62.19%


In [0]:
from sklearn.svm import SVC

In [0]:
svcmodel = SVC(kernel='linear').fit(tout,labels)

In [0]:
pred = svcmodel.predict(tout_test)

In [295]:
cur = 0
for i in range(len(pred)):
    if pred[i] == labels_test[i]:
        cur += 1

print("Accuracy: %.2f%%" % ((cur/float(len(labels_test)) * 100)))

Accuracy: 62.12%


In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
lrmodel = LogisticRegression(solver='lbfgs').fit(tout,labels)

In [0]:
pred = lrmodel.predict(tout_test)

In [299]:
cur = 0
for i in range(len(pred)):
    if pred[i] == labels_test[i]:
        cur += 1

print("Accuracy: %.2f%%" % ((cur/float(len(labels_test)) * 100)))

Accuracy: 62.59%


In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [0]:
nbmodel_pipe = Pipeline([('cvect',CountVectorizer()),
                         ('tfidfvect',TfidfTransformer()),
                         ('nbmodel',MultinomialNB())])

In [302]:
params = {'cvect__ngram_range':[(1,1),(1,2),(1,3)],'tfidfvect__use_idf':(True,False),'nbmodel__alpha':(1e-2,1e-3)} 

gs_nbmodel = GridSearchCV(nbmodel_pipe,params,n_jobs=-1)

gs_nbmodel = gs_nbmodel.fit(texts,labels)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [303]:
gs_nbmodel.best_score_

0.6068205484206872

In [304]:
gs_nbmodel.best_params_

{'cvect__ngram_range': (1, 3),
 'nbmodel__alpha': 0.01,
 'tfidfvect__use_idf': False}

In [0]:
pred = gs_nbmodel.predict(texts_test)

In [306]:
cur = 0
for i in range(len(pred)):
    if pred[i] == labels_test[i]:
        cur += 1

print("Accuracy: %.2f%%" % ((cur/float(len(labels_test)) * 100)))

Accuracy: 60.06%


In [0]:
lrmodel_pipe = Pipeline([('cvect',CountVectorizer()),
                         ('tfidfvect',TfidfTransformer()),
                         ('lrmodel',LogisticRegression(solver = 'lbfgs'))])

In [308]:
params = {'cvect__ngram_range':[(1,1),(1,2),(1,3)],'tfidfvect__use_idf':(True,False)} 

gs_lrmodel = GridSearchCV(lrmodel_pipe,params,n_jobs=-1)

gs_lrmodel = gs_lrmodel.fit(texts,labels)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [309]:
gs_lrmodel.best_score_

0.6292953835473794

In [310]:
gs_lrmodel.best_params_

{'cvect__ngram_range': (1, 2), 'tfidfvect__use_idf': True}

In [0]:
pred = gs_lrmodel.predict(texts_test)

In [312]:
cur = 0
for i in range(len(pred)):
    if pred[i] == labels_test[i]:
        cur += 1

print("Accuracy: %.2f%%" % ((cur/float(len(labels_test)) * 100)))

Accuracy: 62.51%


###Binary Classification using Logistic Regression,Naive Bayes and SVM on statement and justification

In [0]:
df = pd.read_csv(os.getcwd()+'/LIAR-PLUS'+'/dataset'+'/train2.tsv' , sep = '\t' , header = None)
df_val = pd.read_csv(os.getcwd()+'/LIAR-PLUS'+'/dataset'+'/val2.tsv' , sep = '\t' , header = None)
df_test = pd.read_csv(os.getcwd()+'/LIAR-PLUS'+'/dataset'+'/test2.tsv' , sep = '\t' , header = None)

In [0]:
df = df.drop([0,1,4,5,6,7,8,9,10,11,12,13,14],axis=1)

df_val = df_val.drop([0,1,4,5,6,7,8,9,10,11,12,13,14],axis=1)

df_test = df_test.drop([0,1,4,5,6,7,8,9,10,11,12,13,14],axis=1)

df.loc[(df[2] == 'true') , 2] = 5
df.loc[(df[2] == 'mostly-true') , 2] = 4
df.loc[(df[2] == 'half-true') , 2] = 3
df.loc[(df[2] == 'barely-true') , 2] = 2
df.loc[(df[2] == 'false') , 2] = 1
df.loc[(df[2] == 'pants-fire') , 2] = 0
df.columns = ['label','text','just']

df['label'] = (df['label'] > 2).astype(int)

df_val.loc[(df_val[2] == 'true') , 2] = 5
df_val.loc[(df_val[2] == 'mostly-true') , 2] = 4
df_val.loc[(df_val[2] == 'half-true') , 2] = 3
df_val.loc[(df_val[2] == 'barely-true') , 2] = 2
df_val.loc[(df_val[2] == 'false') , 2] = 1
df_val.loc[(df_val[2] == 'pants-fire') , 2] = 0
df_val.columns = ['label','text','just']

df_val['label'] = (df_val['label'] > 2).astype(int)

df_test.loc[(df_test[2] == 'true') , 2] = 5
df_test.loc[(df_test[2] == 'mostly-true') , 2] = 4
df_test.loc[(df_test[2] == 'half-true') , 2] = 3
df_test.loc[(df_test[2] == 'barely-true') , 2] = 2
df_test.loc[(df_test[2] == 'false') , 2] = 1
df_test.loc[(df_test[2] == 'pants-fire') , 2] = 0
df_test.columns = ['label','text','just']

df_test['label'] = (df_test['label'] > 2).astype(int)

df = pd.concat([df,df_val])

In [0]:
labels = df['label'].tolist()
labels = np.array(labels)
labels

labels_val = df_val['label'].tolist()
labels_val = np.array(labels_val)

labels_test = df_test['label'].tolist()
labels_test = np.array(labels_test)

In [0]:
# df['just'] = df['just'].combine_first(df['text'])
# df_val['just'] = df_val['just'].combine_first(df_val['text'])
# df.head()

In [0]:
df['concat'] = df['text'] + '+' + df['just']
df_val['concat'] = df_val['text'] + ' + ' + df_val['just']
df_test['concat'] = df_test['text'] + ' + ' + df_test['just']


In [318]:
df['concat'] = df['concat'].combine_first(df['text'])
df_val['concat'] = df_val['concat'].combine_first(df_val['text'])
df_test['concat'] = df_test['concat'].combine_first(df_test['text'])
df.head()

,label,text,just,concat
0,0,Says the Annies List political group supports ...,That's a premise that he fails to back up. Ann...,Says the Annies List political group supports ...
1,1,When did the decline of coal start? It started...,"Surovell said the decline of coal ""started whe...",When did the decline of coal start? It started...
2,1,"Hillary Clinton agrees with John McCain ""by vo...",Obama said he would have voted against the ame...,"Hillary Clinton agrees with John McCain ""by vo..."
3,0,Health care reform legislation is likely to ma...,The release may have a point that Mikulskis co...,Health care reform legislation is likely to ma...
4,1,The economic turnaround started at the end of ...,"Crist said that the economic ""turnaround start...",The economic turnaround started at the end of ...


In [319]:
df['concat'][0]

0    Says the Annies List political group supports ...
0    We have less Americans working now than in the...
Name: concat, dtype: object

In [0]:
# texts = df['just'].tolist()
# texts_val = df_val['just'].tolist()

# cvect = CountVectorizer()
# cout = cvect.fit_transform(texts)
# cout_val = cvect.transform(texts_val)

# tfidfvect = TfidfTransformer()
# tout1 = tfidfvect.fit_transform(cout)
# tout_val1 = tfidfvect.transform(cout_val)


In [0]:
texts = df['concat'].tolist()
texts_val = df_val['concat'].tolist()
texts_test = df_test['concat'].tolist()

cvect = CountVectorizer()
cout = cvect.fit_transform(texts)
cout_val = cvect.transform(texts_val)
cout_test = cvect.transform(texts_test)

tfidfvect = TfidfTransformer()
tout = tfidfvect.fit_transform(cout)
tout_val = tfidfvect.transform(cout_val)
tout_test = tfidfvect.transform(cout_test)

# tout = cout
# tout_val = cout_val

In [0]:
lrmodel = LogisticRegression(solver='lbfgs').fit(tout,labels)

In [0]:
pred = lrmodel.predict(tout_test)

In [323]:
cur = 0
for i in range(len(pred)):
    if pred[i] == labels_test[i]:
        cur += 1

print("Accuracy: %.2f%%" % ((cur/float(len(labels_test)) * 100)))

Accuracy: 59.51%


In [0]:
svcmodel = SVC(kernel='linear').fit(tout,labels)

In [0]:
pred = svcmodel.predict(tout_test)

In [326]:
cur = 0
for i in range(len(pred)):
    if pred[i] == labels_test[i]:
        cur += 1

print("Accuracy: %.2f%%" % ((cur/float(len(labels_test)) * 100)))

Accuracy: 59.67%


In [0]:
nbmodel = MultinomialNB().fit(tout,labels)

In [0]:
pred = nbmodel.predict(tout_test)

In [329]:
cur = 0
for i in range(len(pred)):
    if pred[i] == labels_test[i]:
        cur += 1

print("Accuracy: %.2f%%" % ((cur/float(len(labels_test)) * 100)))

Accuracy: 57.54%


###6-way Classification using Logistic regression,Naive Bayes and SVM only on statement

In [0]:
df = pd.read_csv(os.getcwd()+'/LIAR-PLUS'+'/dataset'+'/train2.tsv' , sep = '\t' , header = None)
df_val = pd.read_csv(os.getcwd()+'/LIAR-PLUS'+'/dataset'+'/val2.tsv' , sep = '\t' , header = None)
df_test = pd.read_csv(os.getcwd()+'/LIAR-PLUS'+'/dataset'+'/test2.tsv' , sep = '\t' , header = None)

In [0]:
df = df.drop([0,1,4,5,6,7,8,9,10,11,12,13,14,15],axis=1)

df_val = df_val.drop([0,1,4,5,6,7,8,9,10,11,12,13,14,15],axis=1)

df_test = df_test.drop([0,1,4,5,6,7,8,9,10,11,12,13,14,15],axis=1)

df.loc[(df[2] == 'true') , 2] = 5
df.loc[(df[2] == 'mostly-true') , 2] = 4
df.loc[(df[2] == 'half-true') , 2] = 3
df.loc[(df[2] == 'barely-true') , 2] = 2
df.loc[(df[2] == 'false') , 2] = 1
df.loc[(df[2] == 'pants-fire') , 2] = 0
df.columns = ['label','text']

# df['label'] = (df['label'] > 2).astype(int)

df_val.loc[(df_val[2] == 'true') , 2] = 5
df_val.loc[(df_val[2] == 'mostly-true') , 2] = 4
df_val.loc[(df_val[2] == 'half-true') , 2] = 3
df_val.loc[(df_val[2] == 'barely-true') , 2] = 2
df_val.loc[(df_val[2] == 'false') , 2] = 1
df_val.loc[(df_val[2] == 'pants-fire') , 2] = 0
df_val.columns = ['label','text']

# df_val['label'] = (df_val['label'] > 2).astype(int)

df_test.loc[(df_test[2] == 'true') , 2] = 5
df_test.loc[(df_test[2] == 'mostly-true') , 2] = 4
df_test.loc[(df_test[2] == 'half-true') , 2] = 3
df_test.loc[(df_test[2] == 'barely-true') , 2] = 2
df_test.loc[(df_test[2] == 'false') , 2] = 1
df_test.loc[(df_test[2] == 'pants-fire') , 2] = 0
df_test.columns = ['label','text']

# df_test['label'] = (df_test['label'] > 2).astype(int)
df = pd.concat([df,df_val])

In [0]:
labels = df['label'].tolist()
labels = np.array(labels)
labels

labels_val = df_val['label'].tolist()
labels_val = np.array(labels_val)


labels_test = df_test['label'].tolist()
labels_test = np.array(labels_test)

In [0]:
texts = df['text'].tolist()
texts_val = df_val['text'].tolist()
texts_test = df_test['text'].tolist()

cvect = CountVectorizer()
cout = cvect.fit_transform(texts)
cout_val = cvect.transform(texts_val)
cout_test = cvect.transform(texts_test)

tfidfvect = TfidfTransformer()
tout = tfidfvect.fit_transform(cout)
tout_val = tfidfvect.transform(cout_val)
tout_test = tfidfvect.transform(cout_test)

# tout = cout
# tout_val = cout_val

In [334]:
lrmodel = LogisticRegression(solver='lbfgs').fit(tout,labels)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [0]:
pred = lrmodel.predict(tout_test)

In [336]:
cur = 0
for i in range(len(pred)):
    if pred[i] == labels_test[i]:
        cur += 1

print("Accuracy: %.2f%%" % ((cur/float(len(labels_test)) * 100)))

Accuracy: 25.65%


In [0]:
nbmodel = MultinomialNB().fit(tout,labels)

In [0]:
pred = nbmodel.predict(tout_test)

In [339]:
cur = 0
for i in range(len(pred)):
    if pred[i] == labels_test[i]:
        cur += 1

print("Accuracy: %.2f%%" % ((cur/float(len(labels_test)) * 100)))

Accuracy: 26.12%


In [0]:
svcmodel = SVC(kernel='linear').fit(tout,labels)

In [0]:
pred = svcmodel.predict(tout_test)

In [342]:
cur = 0
for i in range(len(pred)):
    if pred[i] == labels_test[i]:
        cur += 1

print("Accuracy: %.2f%%" % ((cur/float(len(labels_test)) * 100)))

Accuracy: 25.89%


###6-Way Classification using Logistic Regression,Naive Bayes and SVM on statement and Justification

In [0]:
df = pd.read_csv(os.getcwd()+'/LIAR-PLUS'+'/dataset'+'/train2.tsv' , sep = '\t' , header = None)
df_val = pd.read_csv(os.getcwd()+'/LIAR-PLUS'+'/dataset'+'/val2.tsv' , sep = '\t' , header = None)
df_test = pd.read_csv(os.getcwd()+'/LIAR-PLUS'+'/dataset'+'/test2.tsv' , sep = '\t' , header = None)

In [0]:
df = df.drop([0,1,4,5,6,7,8,9,10,11,12,13,14],axis=1)

df_val = df_val.drop([0,1,4,5,6,7,8,9,10,11,12,13,14],axis=1)

df_test = df_test.drop([0,1,4,5,6,7,8,9,10,11,12,13,14],axis=1)

df.loc[(df[2] == 'true') , 2] = 5
df.loc[(df[2] == 'mostly-true') , 2] = 4
df.loc[(df[2] == 'half-true') , 2] = 3
df.loc[(df[2] == 'barely-true') , 2] = 2
df.loc[(df[2] == 'false') , 2] = 1
df.loc[(df[2] == 'pants-fire') , 2] = 0
df.columns = ['label','text','just']

# df['label'] = (df['label'] > 2).astype(int)

df_val.loc[(df_val[2] == 'true') , 2] = 5
df_val.loc[(df_val[2] == 'mostly-true') , 2] = 4
df_val.loc[(df_val[2] == 'half-true') , 2] = 3
df_val.loc[(df_val[2] == 'barely-true') , 2] = 2
df_val.loc[(df_val[2] == 'false') , 2] = 1
df_val.loc[(df_val[2] == 'pants-fire') , 2] = 0
df_val.columns = ['label','text','just']

# df_val['label'] = (df_val['label'] > 2).astype(int)

df_test.loc[(df_test[2] == 'true') , 2] = 5
df_test.loc[(df_test[2] == 'mostly-true') , 2] = 4
df_test.loc[(df_test[2] == 'half-true') , 2] = 3
df_test.loc[(df_test[2] == 'barely-true') , 2] = 2
df_test.loc[(df_test[2] == 'false') , 2] = 1
df_test.loc[(df_test[2] == 'pants-fire') , 2] = 0
df_test.columns = ['label','text','just']

# df_test['label'] = (df_test['label'] > 2).astype(int)
df = pd.concat([df,df_val])

In [0]:
labels = df['label'].tolist()
labels = np.array(labels)
labels

labels_val = df_val['label'].tolist()
labels_val = np.array(labels_val)

labels_test = df_test['label'].tolist()
labels_test = np.array(labels_test)

In [0]:
df['concat'] = df['text'] + '+' + df['just']
df_val['concat'] = df_val['text'] + ' + ' + df_val['just']
df_test['concat'] = df_test['text'] + ' + ' + df_test['just']


In [347]:
df['concat'] = df['concat'].combine_first(df['text'])
df_val['concat'] = df_val['concat'].combine_first(df_val['text'])
df_test['concat'] = df_test['concat'].combine_first(df_test['text'])
df.head()

,label,text,just,concat
0,1,Says the Annies List political group supports ...,That's a premise that he fails to back up. Ann...,Says the Annies List political group supports ...
1,3,When did the decline of coal start? It started...,"Surovell said the decline of coal ""started whe...",When did the decline of coal start? It started...
2,4,"Hillary Clinton agrees with John McCain ""by vo...",Obama said he would have voted against the ame...,"Hillary Clinton agrees with John McCain ""by vo..."
3,1,Health care reform legislation is likely to ma...,The release may have a point that Mikulskis co...,Health care reform legislation is likely to ma...
4,3,The economic turnaround started at the end of ...,"Crist said that the economic ""turnaround start...",The economic turnaround started at the end of ...


In [0]:
texts = df['concat'].tolist()
texts_val = df_val['concat'].tolist()
texts_test = df_test['concat'].tolist()

cvect = CountVectorizer()
cout = cvect.fit_transform(texts)
cout_val = cvect.transform(texts_val)
cout_test = cvect.transform(texts_test)

tfidfvect = TfidfTransformer()
tout = tfidfvect.fit_transform(cout)
tout_val = tfidfvect.transform(cout_val)
tout_test = tfidfvect.transform(cout_test)

# tout = cout
# tout_val = cout_val

In [349]:
lrmodel = LogisticRegression(solver='lbfgs').fit(tout,labels)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [0]:
pred = lrmodel.predict(tout_test)

In [351]:
cur = 0
for i in range(len(pred)):
    if pred[i] == labels_test[i]:
        cur += 1

print("Accuracy: %.2f%%" % ((cur/float(len(labels_test)) * 100)))

Accuracy: 23.99%


In [0]:
nbmodel = MultinomialNB().fit(tout,labels)

In [0]:
pred = nbmodel.predict(tout_test)

In [354]:
cur = 0
for i in range(len(pred)):
    if pred[i] == labels_test[i]:
        cur += 1

print("Accuracy: %.2f%%" % ((cur/float(len(labels_test)) * 100)))

Accuracy: 22.10%


In [0]:
svcmodel = SVC(kernel='linear').fit(tout,labels)

In [0]:
pred = svcmodel.predict(tout_test)

In [357]:
cur = 0
for i in range(len(pred)):
    if pred[i] == labels_test[i]:
        cur += 1

print("Accuracy: %.2f%%" % ((cur/float(len(labels_test)) * 100)))

Accuracy: 23.28%


In [154]:
!pip install gensim

###Word Embedding used for Binary Classification

In [0]:
import gensim

In [0]:
df = pd.read_csv(os.getcwd()+'/LIAR-PLUS'+'/dataset'+'/train2.tsv' , sep = '\t' , header = None)
df_val = pd.read_csv(os.getcwd()+'/LIAR-PLUS'+'/dataset'+'/val2.tsv' , sep = '\t' , header = None)
df_test = pd.read_csv(os.getcwd()+'/LIAR-PLUS'+'/dataset'+'/test2.tsv' , sep = '\t' , header = None)

In [0]:
df = df.drop([0,1,4,5,6,7,8,9,10,11,12,13,14,15],axis=1)

df_val = df_val.drop([0,1,4,5,6,7,8,9,10,11,12,13,14,15],axis=1)

df_test = df_test.drop([0,1,4,5,6,7,8,9,10,11,12,13,14,15],axis=1)

df.loc[(df[2] == 'true') , 2] = 5
df.loc[(df[2] == 'mostly-true') , 2] = 4
df.loc[(df[2] == 'half-true') , 2] = 3
df.loc[(df[2] == 'barely-true') , 2] = 2
df.loc[(df[2] == 'false') , 2] = 1
df.loc[(df[2] == 'pants-fire') , 2] = 0
df.columns = ['label','text']

df['label'] = (df['label'] > 2).astype(int)

df_val.loc[(df_val[2] == 'true') , 2] = 5
df_val.loc[(df_val[2] == 'mostly-true') , 2] = 4
df_val.loc[(df_val[2] == 'half-true') , 2] = 3
df_val.loc[(df_val[2] == 'barely-true') , 2] = 2
df_val.loc[(df_val[2] == 'false') , 2] = 1
df_val.loc[(df_val[2] == 'pants-fire') , 2] = 0
df_val.columns = ['label','text']

df_val['label'] = (df_val['label'] > 2).astype(int)

df_test.loc[(df_test[2] == 'true') , 2] = 5
df_test.loc[(df_test[2] == 'mostly-true') , 2] = 4
df_test.loc[(df_test[2] == 'half-true') , 2] = 3
df_test.loc[(df_test[2] == 'barely-true') , 2] = 2
df_test.loc[(df_test[2] == 'false') , 2] = 1
df_test.loc[(df_test[2] == 'pants-fire') , 2] = 0
df_test.columns = ['label','text']

df_test['label'] = (df_test['label'] > 2).astype(int)
df = pd.concat([df,df_val])


In [0]:
labels = df['label'].tolist()
labels = np.array(labels)
labels

labels_val = df_val['label'].tolist()
labels_val = np.array(labels_val)


labels_test = df_test['label'].tolist()
labels_test = np.array(labels_test)

In [0]:
texts = df['text'].tolist()
texts_val = df_val['text'].tolist()
texts_test = df_test['text'].tolist()


In [0]:
word2vec = gensim.models.Word2Vec(texts,size = 200, window = 10, min_count = 2, iter = 10)

In [0]:
def embed(text):
  text = [word for word in text if word in word2vec.wv.vocab]
  return np.mean(word2vec[text],axis = 0)

In [365]:
df['tout'] = df.text.apply(embed)
df_test['tout'] = df_test.text.apply(embed)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
X = list(df['tout'])
X_test = list(df_test['tout'])


In [0]:
lrmodel = LogisticRegression(solver='lbfgs',max_iter = 200).fit(X,labels)

In [0]:
pred = lrmodel.predict(X_test)

In [369]:
cur = 0
for i in range(len(pred)):
    if pred[i] == labels_test[i]:
        cur += 1

print("Accuracy: %.2f%%" % ((cur/float(len(labels_test)) * 100)))

Accuracy: 59.51%


In [0]:
svcmodel = SVC(kernel='linear').fit(X,labels)

In [0]:
pred1 = svcmodel.predict(X_test)

In [372]:
cur = 0
for i in range(len(pred)):
    if pred1[i] == labels_test[i]:
        cur += 1

print("Accuracy: %.2f%%" % ((cur/float(len(labels_test)) * 100)))

Accuracy: 57.93%
